In this notebook, our aim is to examine the dynamics of the relationship between economic growth and CO2 emissions in 27 EU member states in a panel setting for 2000-2017. This is inspired by this paper: https://www.frontiersin.org/articles/10.3389/fenvs.2022.934885/full


In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [2]:
import requests
from io import BytesIO
from zipfile import ZipFile
url = 'https://databank.worldbank.org/data/download/WDI_CSV.zip'
r = requests.get(url, allow_redirects=True)
file = ZipFile(BytesIO(r.content))
data = pd.read_csv(file.open('WDIData.csv'))

Download data from World Bank - World Development Indicators
https://datacatalog.worldbank.org/search/dataset/0037712/World-Development-Indicators

In [3]:
data.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
0,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,17.196986,17.597176,18.034249,18.345878,18.695306,19.149942,19.501837,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,6.580066,6.786218,6.941323,7.096843,7.254828,7.460783,7.599289,NaN,NaN,NaN
2,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.UR.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,37.857526,38.204173,38.303515,38.421813,38.482409,38.692053,38.793983,NaN,NaN,NaN
3,Africa Eastern and Southern,AFE,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,31.824950,33.744405,38.733352,40.092163,42.880977,44.073912,45.609604,NaN,NaN,NaN
4,Africa Eastern and Southern,AFE,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,17.485006,16.329765,24.372504,25.153292,27.227391,29.383000,30.163364,NaN,NaN,NaN


To measure the increase in the
production of goods and services in EU economies, we use the
most common indicator GDP Per Capita (constant U.S. $).
Additionally, the growth process requires energy consumption
and leads to rising atmospheric concentrations of carbon dioxide,
that’s why we include in the analysis the status of carbon
emissions, measured by CO2 Emissions (metric tons per
capita)

In [4]:
df = data.loc[(data['Indicator Code'] == 'NY.GDP.PCAP.KD') |  (data['Indicator Code'] == 'EN.ATM.CO2E.PC')]

In [5]:
df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
193,Africa Eastern and Southern,AFE,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,NaN,NaN,NaN,NaN,NaN,NaN,...,1.006890,0.956693,0.938560,0.928488,0.908127,0.903800,NaN,NaN,NaN,NaN
477,Africa Eastern and Southern,AFE,GDP per capita (constant 2015 US$),NY.GDP.PCAP.KD,1176.014714,1148.259672,1206.960217,1235.275902,1256.624574,1287.016686,...,1537.159571,1538.552268,1534.924746,1534.683482,1533.524108,1523.775055,1438.876104,1464.047225,NaN,NaN
1671,Africa Western and Central,AFW,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,NaN,NaN,NaN,NaN,NaN,NaN,...,0.506868,0.485412,0.490803,0.474887,0.478275,0.485868,NaN,NaN,NaN,NaN
1955,Africa Western and Central,AFW,GDP per capita (constant 2015 US$),NY.GDP.PCAP.KD,1086.567460,1083.580309,1100.841677,1155.696507,1191.667752,1212.810710,...,1876.224943,1876.623483,1829.131551,1821.996368,1826.910949,1836.823853,1773.886822,1797.960202,NaN,NaN
3149,Arab World,ARB,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,NaN,NaN,NaN,NaN,NaN,NaN,...,4.427319,4.446687,4.403927,4.361316,4.222125,4.247838,NaN,NaN,NaN,NaN


We can see from the above that carbon emissions data is not available in early part of the data series. We could run the code below to filter out the ones with more than 10% NA.

In [6]:
df = df.dropna(thresh= len(df) * 0.9, axis = 1)
df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,2000,2001,2002,2003,2004,2005,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
193,Africa Eastern and Southern,AFE,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,0.886296,0.954219,0.956154,0.982868,1.027650,1.002883,...,1.011862,0.970206,0.983517,0.995680,1.006890,0.956693,0.938560,0.928488,0.908127,0.903800
477,Africa Eastern and Southern,AFE,GDP per capita (constant 2015 US$),NY.GDP.PCAP.KD,1264.526699,1277.632516,1293.803019,1299.833172,1336.080180,1380.986752,...,1512.484129,1526.646519,1497.369205,1518.482850,1537.159571,1538.552268,1534.924746,1534.683482,1533.524108,1523.775055
1671,Africa Western and Central,AFW,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,0.525693,0.539231,0.497096,0.515326,0.501134,0.501195,...,0.464460,0.468731,0.472266,0.499202,0.506868,0.485412,0.490803,0.474887,0.478275,0.485868
1955,Africa Western and Central,AFW,GDP per capita (constant 2015 US$),NY.GDP.PCAP.KD,1191.155978,1219.173837,1303.684669,1338.118134,1405.710653,1446.876559,...,1690.634529,1723.563611,1762.502106,1819.906214,1876.224943,1876.623483,1829.131551,1821.996368,1826.910949,1836.823853
3149,Arab World,ARB,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,3.252753,3.357020,3.396012,3.423837,3.531226,3.686096,...,4.206228,4.205058,4.400961,4.378777,4.427319,4.446687,4.403927,4.361316,4.222125,4.247838


Only extract 27 countries from EU

In [7]:
EU27 = ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czechia', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovak Republic', 'Slovenia', 'Spain', 'Sweden']
filtered_df = df[df['Country Name'].isin(EU27)]
filtered_df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,2000,2001,2002,2003,2004,2005,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
88873,Austria,AUT,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,7.930073,8.444271,8.583057,9.168541,9.275100,9.265701,...,8.365015,8.135654,7.723270,7.753346,7.260276,7.316962,7.289335,7.485741,7.133279,7.293568
89157,Austria,AUT,GDP per capita (constant 2015 US$),NY.GDP.PCAP.KD,38842.890520,39184.808597,39636.482611,39815.221953,40651.226616,41281.270756,...,43334.508964,44451.000192,44549.881698,44299.378185,44245.168740,44195.817595,44590.251628,45281.723400,46154.625096,46647.080963
99219,Belgium,BEL,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,11.440029,11.504301,10.728008,11.131517,10.941484,10.555859,...,9.795003,8.740704,8.577526,8.655146,8.041470,8.438012,8.315170,8.151430,8.196365,8.095453
99503,Belgium,BEL,GDP per capita (constant 2015 US$),NY.GDP.PCAP.KD,35674.791253,35943.237960,36393.241767,36617.380384,37761.281319,38426.051908,...,39777.925277,39929.095144,39975.573640,39970.317497,40421.420792,41008.296719,41318.019639,41825.762832,42382.318276,43098.484720
115477,Bulgaria,BGR,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,5.314588,5.766610,5.563226,6.164795,6.124991,6.265279,...,6.049625,6.754952,6.162577,5.459060,5.821534,6.207378,5.834419,6.201968,5.821777,5.611302


In [8]:
# select only these two columns and years
filtered_df = filtered_df.loc[:, ['Country Name','Indicator Name'] + list(filtered_df.columns[4:])]

We then use melt and pivot table to convert the data frame into the format below to prepare for regression

In [9]:
df_new = filtered_df.melt(id_vars = ['Country Name', 'Indicator Name'], var_name = 'Year', value_name = 'Value')
df_new = df_new.pivot_table(index = ['Country Name', 'Year'], columns = 'Indicator Name', values = 'Value').reset_index()
df_new = df_new.rename_axis(index = None, columns = None)
df_new

,Country Name,Year,CO2 emissions (metric tons per capita),GDP per capita (constant 2015 US$)
0,Austria,2000,7.930073,38842.890520
1,Austria,2001,8.444271,39184.808597
2,Austria,2002,8.583057,39636.482611
3,Austria,2003,9.168541,39815.221953
4,Austria,2004,9.275100,40651.226616
...,...,...,...,...
535,Sweden,2015,3.999557,51545.483610
536,Sweden,2016,3.908301,51955.861070
537,Sweden,2017,3.807213,52576.810282
538,Sweden,2018,3.539297,52983.006862


In [10]:
# Create new data frame with correct data type
data_frame = {'Country': df_new['Country Name'].astype(str),
              'Year': df_new['Year'].astype(int),
              'CO2_emissions': df_new['CO2 emissions (metric tons per capita)'].astype(float),
              'GDP': df_new['GDP per capita (constant 2015 US$)'].astype(float)}
df = pd.DataFrame(data_frame)

# Sort data by country and year
df_sorted = df.sort_values(['Country', 'Year'])

# Compute first differences of the variables 
df_diff = df_sorted.groupby('Country').diff().dropna().reset_index()

## CO2 emissions per capita versus GDP per capita (level)

In [11]:
# Run OLS for df sorted - look at LEVEL
model_level = sm.OLS(df_sorted['CO2_emissions'], sm.add_constant(df_sorted['GDP']))

# Fit the model
results = model_level.fit()

# Print the model summary
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:          CO2_emissions   R-squared:                       0.434
Model:                            OLS   Adj. R-squared:                  0.433
Method:                 Least Squares   F-statistic:                     412.9
Date:                Sun, 18 Jun 2023   Prob (F-statistic):           1.49e-68
Time:                        12:54:20   Log-Likelihood:                -1295.3
No. Observations:                 540   AIC:                             2595.
Df Residuals:                     538   BIC:                             2603.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.3496      0.197     22.040      0.0

## CO2 emissions per capita versus GDP per capita (first difference)

In [12]:
# Run OLS for df sorted - look at LEVEL
model_diff = sm.OLS(df_diff['CO2_emissions'], sm.add_constant(df_diff['GDP']))

# Fit the model
results = model_diff.fit()

# Print the model summary
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:          CO2_emissions   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     22.44
Date:                Sun, 18 Jun 2023   Prob (F-statistic):           2.81e-06
Time:                        12:54:20   Log-Likelihood:                -415.47
No. Observations:                 513   AIC:                             834.9
Df Residuals:                     511   BIC:                             843.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1402      0.026     -5.390      0.0

## CO2 emissions per capita versus GDP per capita (include lagged values)

In [13]:
# Add lagged variables to the dataframe - to see if lagged values have any predictive power
df_lagged = df_diff.join(df_sorted[['CO2_emissions', 'GDP']].groupby(df_sorted['Country']).shift(), rsuffix='_lag').dropna()
df_lagged

,index,Year,CO2_emissions,GDP,CO2_emissions_lag,GDP_lag
1,2,1.0,0.138786,451.674014,7.930073,38842.890520
2,3,1.0,0.585484,178.739342,8.444271,39184.808597
3,4,1.0,0.106559,836.004662,8.583057,39636.482611
4,5,1.0,-0.009399,630.044140,9.168541,39815.221953
5,6,1.0,-0.304216,1215.080296,9.275100,40651.226616
...,...,...,...,...,...,...
508,535,1.0,-0.031400,1690.182597,7.842321,27218.449260
509,536,1.0,-0.091256,410.377461,7.056479,27025.294943
510,537,1.0,-0.101088,620.949212,6.200864,25778.956465
511,538,1.0,-0.267917,406.196580,5.866733,25702.346845


In [14]:
# Specify the model
model_lagged = sm.OLS(df_lagged['CO2_emissions'], sm.add_constant(df_lagged[['CO2_emissions_lag', 'GDP', 'GDP_lag']]))

# Fit the model
results = model_lagged.fit()

# Print the model summary
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:          CO2_emissions   R-squared:                       0.047
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     7.854
Date:                Sun, 18 Jun 2023   Prob (F-statistic):           3.98e-05
Time:                        12:54:20   Log-Likelihood:                -399.56
No. Observations:                 487   AIC:                             807.1
Df Residuals:                     483   BIC:                             823.9
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -0.0647      0.06

## CO2 emissions per capita versus GDP per capita (add country dummies)

In [15]:
df_new = df_new.set_index(['Country Name'])
dummies = pd.get_dummies(df_new.index.get_level_values(0))
# important - need to replace space with _ or else will get syntax error
dummies.columns = dummies.columns.str.replace(" ","_")



In [16]:
formula = 'CO2_emissions ~ GDP + ' + '+'.join(dummies.columns)

In [17]:
formula

'CO2_emissions ~ GDP + Austria+Belgium+Bulgaria+Croatia+Cyprus+Czechia+Denmark+Estonia+Finland+France+Germany+Greece+Hungary+Ireland+Italy+Latvia+Lithuania+Luxembourg+Malta+Netherlands+Poland+Portugal+Romania+Slovak_Republic+Slovenia+Spain+Sweden'

In [18]:
dummies

,Austria,Belgium,Bulgaria,Croatia,Cyprus,Czechia,Denmark,Estonia,Finland,France,...,Luxembourg,Malta,Netherlands,Poland,Portugal,Romania,Slovak_Republic,Slovenia,Spain,Sweden
0,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
536,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
537,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
538,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [19]:
OLS_df = pd.concat([df_sorted, dummies], axis=1) # combine into one data
OLS_df = OLS_df.set_index(['Country', 'Year'])


In [20]:
# Fit the model using OLS
model = sm.formula.ols(formula=formula, data=OLS_df)
results = model.fit()

# View the regression results
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:          CO2_emissions   R-squared:                       0.902
Model:                            OLS   Adj. R-squared:                  0.896
Method:                 Least Squares   F-statistic:                     173.8
Date:                Sun, 18 Jun 2023   Prob (F-statistic):          4.24e-238
Time:                        12:54:20   Log-Likelihood:                -822.99
No. Observations:                 540   AIC:                             1702.
Df Residuals:                     512   BIC:                             1822.
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

## CO2 emissions per capita versus GDP per capita (percentage growth) 

In [21]:
# Group the data by country and apply pct_change to calculate GDP growth percentage
grouped = OLS_df.groupby('Country')
OLS_df['gdp_growth'] = grouped['GDP'].pct_change() * 100
OLS_df['CO2_emissions_growth'] = grouped['CO2_emissions'].pct_change() * 100


In [22]:
formula = 'CO2_emissions_growth ~ gdp_growth + GDP + ' + '+'.join(dummies.columns)
formula

'CO2_emissions_growth ~ gdp_growth + GDP + Austria+Belgium+Bulgaria+Croatia+Cyprus+Czechia+Denmark+Estonia+Finland+France+Germany+Greece+Hungary+Ireland+Italy+Latvia+Lithuania+Luxembourg+Malta+Netherlands+Poland+Portugal+Romania+Slovak_Republic+Slovenia+Spain+Sweden'

In [23]:
# Fit the model using OLS
model = sm.formula.ols(formula=formula, data=OLS_df)
results = model.fit()

# View the regression results
print(results.summary())

                             OLS Regression Results                             
Dep. Variable:     CO2_emissions_growth   R-squared:                       0.183
Model:                              OLS   Adj. R-squared:                  0.136
Method:                   Least Squares   F-statistic:                     3.870
Date:                  Sun, 18 Jun 2023   Prob (F-statistic):           4.45e-10
Time:                          12:54:20   Log-Likelihood:                -1599.3
No. Observations:                   513   AIC:                             3257.
Df Residuals:                       484   BIC:                             3380.
Df Model:                            28                                         
Covariance Type:              nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Interc

We did find some positive relationship between gdp_growth and carbon emissions growth with a significant t-stat. On average, a 1% change in GDP leads to a 0.65% change in CO2 emissions

In [24]:
formula2 = 'CO2_emissions_growth ~ gdp_growth + GDP' #remove country fixed effects
# Fit the model using OLS
model = sm.formula.ols(formula=formula2, data=OLS_df)
results = model.fit()

# View the regression results
print(results.summary())

                             OLS Regression Results                             
Dep. Variable:     CO2_emissions_growth   R-squared:                       0.131
Model:                              OLS   Adj. R-squared:                  0.127
Method:                   Least Squares   F-statistic:                     38.38
Date:                  Sun, 18 Jun 2023   Prob (F-statistic):           2.97e-16
Time:                          12:54:20   Log-Likelihood:                -1615.2
No. Observations:                   513   AIC:                             3236.
Df Residuals:                       510   BIC:                             3249.
Df Model:                             2                                         
Covariance Type:              nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.0164      0.492

## [OPTIONAL] Extending the analysis to examine high income versus low income

In [25]:
group = ['Low income', 'OECD members']
filtered_data2 = data[data['Country Name'].isin(group)]
filtered_data2.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
41384,Low income,LIC,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,8.434338,8.758202,9.153085,9.475340,9.812145,10.230183,10.583029,NaN,NaN,NaN
41385,Low income,LIC,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,3.253891,3.448386,3.675843,3.892542,4.075566,4.363033,4.541178,NaN,NaN,NaN
41386,Low income,LIC,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.UR.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,19.936333,20.258748,20.503887,20.842016,21.074496,21.461405,21.754747,NaN,NaN,NaN
41387,Low income,LIC,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,29.442365,29.497668,35.010157,36.348114,39.124267,39.701824,41.369165,NaN,NaN,NaN
41388,Low income,LIC,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,17.143172,15.195785,23.833524,24.557024,26.977544,28.026925,28.308368,NaN,NaN,NaN


In [26]:
filtered_data2 = filtered_data2.loc[(filtered_data2['Indicator Code'] == 'NY.GDP.PCAP.KD') |  (filtered_data2['Indicator Code'] == 'EN.ATM.CO2E.PC')]

In [27]:
filtered_data2.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
41577,Low income,LIC,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,NaN,NaN,NaN,NaN,NaN,NaN,...,0.227327,0.223378,0.236358,0.272190,0.271918,0.277174,NaN,NaN,NaN,NaN
41861,Low income,LIC,GDP per capita (constant 2015 US$),NY.GDP.PCAP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,766.258236,775.673373,788.463311,799.352834,806.952688,811.571351,790.912020,789.660253,NaN,NaN
53401,OECD members,OED,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,NaN,NaN,NaN,NaN,NaN,NaN,...,9.148276,9.041205,8.928317,8.854910,8.832567,8.518895,NaN,NaN,NaN,NaN
53685,OECD members,OED,GDP per capita (constant 2015 US$),NY.GDP.PCAP.KD,10921.648301,11238.593473,11707.606753,12166.03418,12775.919452,13297.984854,...,34982.625779,35599.080040,36012.034305,36688.090402,37341.605211,37805.606901,36010.726795,37880.953547,NaN,NaN


In [28]:
# select only these two columns and years
filtered_data2 = filtered_data2.loc[:, ['Country Name','Indicator Name'] + list(filtered_df.columns[4:])]
df_new = filtered_data2.melt(id_vars = ['Country Name', 'Indicator Name'], var_name = 'Year', value_name = 'Value')
df_new = df_new.pivot_table(index = ['Country Name', 'Year'], columns = 'Indicator Name', values = 'Value').reset_index()
df_new = df_new.rename_axis(index = None, columns = None)
df_new

,Country Name,Year,CO2 emissions (metric tons per capita),GDP per capita (constant 2015 US$)
0,Low income,2002,0.293298,638.272877
1,Low income,2003,0.290200,639.949702
2,Low income,2004,0.290178,654.430407
3,Low income,2005,0.297718,673.120678
4,Low income,2006,0.299195,691.958901
5,Low income,2007,0.271838,714.250609
6,Low income,2008,0.283630,727.380378
7,Low income,2009,0.252221,732.512054
8,Low income,2010,0.254462,759.702348
9,Low income,2011,0.235475,763.207350


In [29]:
# Create new data frame with correct data type
data_frame = {'Country': df_new['Country Name'].astype(str),
              'Year': df_new['Year'].astype(int),
              'CO2_emissions': df_new['CO2 emissions (metric tons per capita)'].astype(float),
              'GDP': df_new['GDP per capita (constant 2015 US$)'].astype(float)}
df = pd.DataFrame(data_frame)

# Sort data by country and year
df_sorted = df.sort_values(['Country', 'Year'])
df_sorted['Country'] = df_sorted['Country'].str.replace(' ', '_') # replace blank with underscore :: to prevent syntax issues


In [30]:
dummies_df = df_sorted.set_index(['Country'])
dummies = pd.get_dummies(dummies_df.index.get_level_values(0))


In [31]:
OLS_df2 = pd.concat([df_sorted, dummies], axis=1) # combine df with dummies

In [32]:
# Group the data by country and apply pct_change to calculate GDP growth percentage
grouped = OLS_df2.groupby('Country')
OLS_df2['gdp_growth'] = grouped['GDP'].pct_change() * 100
OLS_df2['CO2_emissions_growth'] = grouped['CO2_emissions'].pct_change() * 100

In [33]:
OLS_df2.rename(columns={'Low income': 'Low_income'}, inplace=True)
OLS_df2.rename(columns={'OECD members': 'OECD_members'}, inplace=True)

In [34]:
formula = 'CO2_emissions ~ gdp_growth + GDP + ' + '+'.join(dummies.columns)
# Fit the model using OLS
model = sm.formula.ols(formula=formula, data=OLS_df2)
results = model.fit()

# View the regression resultsOS
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:          CO2_emissions   R-squared:                       0.997
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                     3785.
Date:                Sun, 18 Jun 2023   Prob (F-statistic):           9.18e-39
Time:                        12:54:20   Log-Likelihood:              -0.040799
No. Observations:                  34   AIC:                             8.082
Df Residuals:                      30   BIC:                             14.19
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                7.4145 

In [35]:
# You can also run analysis on these other countries below 
data['Country Name'].unique()

array(['Africa Eastern and Southern', 'Africa Western and Central',
       'Arab World', 'Caribbean small states',
       'Central Europe and the Baltics', 'Early-demographic dividend',
       'East Asia & Pacific',
       'East Asia & Pacific (excluding high income)',
       'East Asia & Pacific (IDA & IBRD countries)', 'Euro area',
       'Europe & Central Asia',
       'Europe & Central Asia (excluding high income)',
       'Europe & Central Asia (IDA & IBRD countries)', 'European Union',
       'Fragile and conflict affected situations',
       'Heavily indebted poor countries (HIPC)', 'High income',
       'IBRD only', 'IDA & IBRD total', 'IDA blend', 'IDA only',
       'IDA total', 'Late-demographic dividend',
       'Latin America & Caribbean',
       'Latin America & Caribbean (excluding high income)',
       'Latin America & the Caribbean (IDA & IBRD countries)',
       'Least developed countries: UN classification',
       'Low & middle income', 'Low income', 'Lower middle in